## Handling User and Reading list (=Media list) data

#### ToDo

- [x] Favorites are not enough to describe user preferences -> get anime/manga list that the users' have experienced.
- [x] Convert "reading list" (= media list) to list-genre table to express which genre a user has experienced the most
- [x] Try similar approach with tag data
- [ ] Weight the experienced media by its status (CURRENT/DROPPED etc)
- [ ] Average the summed flag values by media count per user -> probability of choosing a particular genre/tag
- [ ] Extra: Convert media list to network visualization

Need to be able to see what kind of genre/type that the user lieks

In [11]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from collections import ChainMap
import sklearn
from sklearn.preprocessing import MultiLabelBinarizer
pd.set_option("max_columns", 200)

In [2]:
# Load data
df_titles = pd.read_csv("../assets/titles_2000p.csv")
df_tags = pd.read_csv("../assets/tags_2000p.csv")

In [3]:
print(df_titles.shape)
print(df_tags.shape)

(98927, 35)
(269425, 7)


### title:tags table. (Commented out to avoid running)

In [4]:
# id_romaji_match = df_titles[["title_id", "title_romaji"]]
# print(id_romaji_match.shape)

# # Convert tags table
# res = []

# for t_id in tqdm(df_tags["title_id"].unique()):
#     df = df_tags[df_tags["title_id"]==t_id]
#     tag_name_list = list(df["tag_name"].unique())
#     title_tags = dict()
#     title_tags[t_id] = str(tag_name_list)
#     res.append(title_tags)

# res_list = []
# for r in res:
#     for k,v in r.items():
#         res_list.append([k,v])

# df_title_tag = pd.DataFrame(res_list, columns=["title_id", "tag_name_array"])
# print(df_title_tag.shape)
# display(df_title_tag.head())

# df_title_tag_romaji = pd.merge(id_romaji_match, df_title_tag, how="inner", on="title_id")
# display(df_title_tag_romaji.head())
# print(df_title_tag_romaji.shape)

# # check id-romaji match
# display(df_titles[df_titles["title_id"]==103948])
# display(df_title_tag_romaji[df_title_tag_romaji["title_id"]==86018])

# # Convert tag_name_array columns to OHE-like columns
# df_title_tag_romaji["tag_name_array"] = df_title_tag_romaji["tag_name_array"].replace("['|\"|\[|\]]", "", regex=True)
# df_title_tag_romaji["tag_name_array"] = df_title_tag_romaji["tag_name_array"].apply(lambda x: x.split(","))
# display(df_title_tag_romaji[df_title_tag_romaji["title_id"]==86018])

# mlb = MultiLabelBinarizer()
# df_title_tag_romaji_ohe = df_title_tag_romaji.join(pd.DataFrame(mlb.fit_transform(df_title_tag_romaji["tag_name_array"]), columns=mlb.classes_))
# df_title_tag_romaji_ohe = df_title_tag_romaji_ohe.drop("tag_name_array", axis=1)
# display(df_title_tag_romaji_ohe.head(2))

# df_title_tag_romaji_ohe.to_csv("../assets/ryota_title_tags_2000p.csv", index=False)

### title:genre table. (Commented out to avoid running)

In [5]:
# # Preprocess genre column: convert from string to array
# df_titles["genres"] = df_titles["genres"].apply(lambda x: re.findall("'(.*?)'", x))

# # Convert genre columns to OHE-like columns
# mlb = MultiLabelBinarizer()
# df_titles = df_titles.join(pd.DataFrame(mlb.fit_transform(df_titles["genres"]),columns=mlb.classes_))
# df_titles = df_titles.drop("genres", axis=1)

# # Create titles_to_genre table for future use
# df_titles_idx = df_titles.set_index("title_id")
# df_titles_genre = df_titles_idx.iloc[:, 33:52].reset_index()
# display(df_titles_genre)
# df_titles_genre.to_csv("../assets/ryota_title_genre_2000p.csv", index=False)

### media_list:genre table (Commented out to avoid running)

In [45]:
# # Work with the titles that the users have read or are reading

# # First, concatenate the separated "media_list" files
# df_mlist_1 = pd.read_csv("../assets/media_list_2000_users.csv")
# df_mlist_2 = pd.read_csv("../assets/media_list_2001_5000_users.csv")
# df_mlist_3 = pd.read_csv("../assets/media_list_5001_10000_users.csv")
# df_mlist = pd.concat([df_mlist_1, df_mlist_2, df_mlist_3], axis=0)
# print("length of media lists before joining: ", len(df_mlist_1), len(df_mlist_2), len(df_mlist_3))
# print("length of concat media list dfs: ", len(df_mlist))

# # Save the concatenated df as csv
# df_mlist.to_csv("../assets/media_list_all_users.csv", index=False)

# # For every title that user chooses, convert it to title-genre to show their preference
# df_titles_genre = pd.read_csv("../assets/ryota_title_genre_2000p.csv")
# df_mlist_genre = df_mlist.join(df_titles_genre, how="left", on="title_id", rsuffix="_overlap")
# df_mlist_genre = df_mlist_genre.drop(columns=["title_id_overlap"])
# print("rows with no genre info: ", len(df_mlist_genre[df_mlist_genre.isna().any(axis=1)]))
# df_mlist_genre = df_mlist_genre.dropna(how="any")
# print("length after dropping NaNs: ", len(df_mlist_genre))

# # Aggregate genre counts per user
# res = []
# for u_id in tqdm(df_mlist_genre["user_id"].unique()):
#     df = df_mlist_genre[df_mlist_genre["user_id"]==u_id]
#     genre_counts = df.iloc[:, 5:].sum(axis=0).values
#     genre_count_row = np.insert(genre_counts, [0,0], [u_id, len(df)])
#     res.append(genre_count_row)

# cols = ["user_id", "mlist_count"]
# tag_cols = list(df_mlist_genre.iloc[:, 5:].columns)
# df_mlist_genre = pd.DataFrame(res, columns=cols+tag_cols)
# display(df_mlist_genre)

# # convert float columns to ints
# for col in df_mlist_genre.columns:
#     if df_mlist_genre[col].dtype == np.float:
#         df_mlist_genre[col] = df_mlist_genre[col].astype(int)

# # Save medialist-genre df as csv
# df_mlist_genre.to_csv("../assets/ryota_media_list_genre.csv", index=False)


### with media_list:tags table. Simple df.join causes MEMORY ERROR so iterate over user_id and title_id

In [46]:
# # Work with the titles that the users have read or are reading

# # Load concat "media_list"
# df_mlist = pd.read_csv("../assets/media_list_all_users.csv")
# print("length of concat media list dfs: ", len(df_mlist))

# # For every title that user chooses, convert it to title-tags to show their preference
# df_titles_tags = pd.read_csv("../assets/ryota_title_tags_2000p.csv")

# print(df_mlist.shape)
# print(df_titles_tags.shape)

# # df.join like mlist-genre doesn't work as it causes MEMORY ERROR.
# # so iterate over user_id and title_id instead
# res = []
# for u_id in tqdm(df_mlist["user_id"].unique()):
#     user_row = []
#     df = df_mlist[df_mlist["user_id"]==u_id]
#     for t_id in df["title_id"].unique():
#         try:
#             df_tags = df_titles_tags[df_titles_tags["title_id"]==t_id]
#             tag_flags = df_tags.values[0][2:]
#             user_row.append(tag_flags)
#         except:
#             pass
#             # print("error at: ", u_id, t_id)
#     sum_row = sum(user_row)
#     sum_row = np.insert(sum_row, [0,0], [u_id, len(user_row)])
#     res.append(sum_row)

# cols = ["user_id", "mlist_count"]
# tag_cols = list(df_titles_tags.iloc[:, 2:].columns)
# df_mlist_tags = pd.DataFrame(res, columns=cols+tag_cols)

# # print("rows with no tag info: ", len(df_mlist_tags[df_mlist_tags.isna().any(axis=1)]))
# # df_mlist_tags = df_mlist_tags.dropna(how="any")
# # print("length after dropping NaNs: ", len(df_mlist_tags))

# # convert to ints
# for col in df_mlist_tags.columns:
#     if df_mlist_tags[col].dtype == np.float:
#         df_mlist_tags[col] = df_mlist_tags[col].astype(int)

# # Save medialist-genre df as csv
# df_mlist_tags.to_csv("../assets/ryota_media_list_tags.csv", index=False)

### Real work

In [44]:
df_media_list_genre = pd.read_csv("../assets/ryota_media_list_genre.csv")
df_media_list_tags = pd.read_csv("../assets/ryota_media_list_tags.csv")
df_media_list_tags = df_media_list_tags.dropna(how="any") # somehow 1 record with NaN wasn't dropped
display(df_media_list_genre)
display(df_media_list_tags)

,user_id,mlist_count,Action,Adventure,Comedy,Drama,Ecchi,Fantasy,Hentai,Horror,Mahou Shoujo,Mecha,Music,Mystery,Psychological,Romance,Sci-Fi,Slice of Life,Sports,Supernatural,Thriller
0,1,361,89,65,133,100,36,88,37,9,9,11,12,25,18,129,30,60,14,42,3
1,2,287,76,49,120,89,22,76,21,11,6,9,6,20,21,114,30,62,15,38,6
2,5,351,83,61,110,114,16,62,32,11,1,15,10,34,33,97,42,48,17,62,7
3,6,10,2,2,2,2,1,2,3,0,0,0,0,0,0,0,0,1,0,0,0
4,7,431,121,83,176,117,40,113,32,19,6,9,6,36,31,166,47,77,17,63,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6709,9968,484,120,87,196,151,38,120,36,17,8,14,9,37,31,162,54,86,25,64,11
6710,9971,245,56,24,92,62,20,56,24,9,5,4,5,17,17,79,25,53,14,35,2
6711,9991,47,8,5,27,9,7,11,5,2,2,1,1,3,3,18,1,8,2,10,0
6712,9992,69,12,9,32,17,10,14,7,1,0,2,2,5,2,23,2,13,3,11,1


,user_id,mlist_count,4-koma,Abuse,Achromatic,Achronological Order,Acting,Adoption,Advertisement,Afterlife,Age Gap,Age Regression,Agender,Agriculture,Ahegao,Airsoft,Aliens,Alternate Universe,American Football,Amnesia,Amputation,Anachronism,Anal Sex,Angels,Animals,Anthology,Anti-Hero,Archery,Armpits,Artificial Intelligence,Asexual,Ashikoki,Asphyxiation,Assassins,Astronomy,Athletics,Augmented Reality,Autobiographical,Aviation,Badminton,Band,Bar,Baseball,Basketball,Battle Royale,Biographical,Bisexual,Blackmail,Body Horror,Body Swapping,Bondage,Boobjob,Boxing,Boys Love,Bullying,Butler,CGI,Calligraphy,Cannibalism,Card Battle,Cars,Centaur,Cheerleading,Chibi,Chimera,Chuunibyou,Circus,Classic Literature,Clone,College,Coming of Age,Conspiracy,Cosmic Horror,Cosplay,Crime,Crossdressing,Crossover,Cult,Cultivation,Cumflation,Cunnilingus,Cute Boys Doing Cute Things,Cute Girls Doing Cute Things,Cyberpunk,Cyborg,Cycling,DILF,Dancing,Death Game,Deepthroat,Defloration,Delinquents,Demons,Denpa,Detective,Dinosaurs,Disability,Dissociative Identities,Dragons,Drawing,...,Primarily Male Cast,Primarily Teen Cast,Prostitution,Public Sex,Puppetry,Rakugo,Rape,Real Robot,Rehabilitation,Reincarnation,Religion,Revenge,Robots,Rotoscoping,Rugby,Rural,Sadism,Samurai,Satire,Scat,School,School Club,Scissoring,Scuba Diving,Seinen,Sex Toys,Shapeshifting,Ships,Shogi,Shoujo,Shounen,Shrine Maiden,Skateboarding,Skeleton,Slapstick,Slavery,Software Development,Space,Space Opera,Steampunk,Stop Motion,Succubus,Suicide,Sumata,Sumo,Super Power,Super Robot,Superhero,Surfing,Surreal Comedy,Survival,Sweat,Swimming,Swordplay,Table Tennis,Tanks,Tanned Skin,Teacher,Teens Love,Tennis,Tentacles,Terrorism,Threesome,Time Manipulation,Time Skip,Tokusatsu,Tomboy,Torture,Tragedy,Trains,Transgender,Travel,Triads,Tsundere,Twins,Urban,Urban Fantasy,VTuber,Vampire,Video Games,Vikings,Villainess,Virginity,Virtual World,Volleyball,Vore,Voyeur,War,Watersports,Werewolf,Witch,Work,Wrestling,Writing,Wuxia,Yakuza,Yandere,Youkai,Yuri,Zombie
0,1,398,0,0,0,30,8,10,0,14,22,11,8,5,0,3,31,35,0,45,0,10,0,5,18,2,55,7,0,24,4,0,0,14,3,4,3,1,6,1,12,6,9,2,13,3,19,0,42,10,0,0,4,2,30,3,60,0,11,1,3,1,0,10,2,18,1,4,0,8,90,22,31,7,34,32,1,14,7,0,0,4,39,13,10,2,0,2,16,0,0,11,34,28,14,1,16,30,15,13,...,0,0,0,0,0,0,0,0,4,1,0,2,3,0,0,0,0,2,3,0,18,4,0,0,8,0,0,0,0,0,10,0,0,0,2,0,0,6,1,0,0,0,1,0,0,15,3,6,0,9,6,0,1,1,0,1,0,0,0,0,0,1,0,6,0,1,1,0,5,0,0,0,0,2,1,0,9,0,4,3,0,0,0,5,0,0,0,2,0,0,1,6,0,1,0,0,0,3,2,2
1,2,303,0,0,0,12,3,10,0,10,11,8,3,2,0,0,30,25,0,32,0,4,0,4,7,0,32,7,0,10,2,0,0,9,1,2,3,0,4,1,14,4,12,3,10,1,7,0,29,14,0,0,3,2,28,7,42,0,9,3,0,0,1,11,4,11,3,3,1,9,63,15,18,6,23,25,4,6,9,0,0,4,23,5,7,1,0,1,11,0,0,12,25,13,12,5,16,24,10,7,...,0,0,0,0,0,0,0,0,1,1,0,0,2,0,0,0,0,0,2,0,17,6,0,0,4,0,0,0,0,1,18,0,0,0,1,0,0,1,0,0,0,0,1,0,0,14,2,4,0,4,4,0,1,2,0,0,0,1,0,0,0,1,0,4,0,0,0,0,6,0,0,0,0,2,0,0,5,0,4,1,0,0,0,2,1,1,0,1,0,0,1,3,0,3,0,0,0,2,1,5
2,5,323,2,0,0,2,7,5,0,8,17,3,2,1,0,0,19,8,0,21,0,8,0,3,4,1,16,1,0,8,0,0,0,13,0,3,1,0,3,0,4,2,8,4,3,0,4,0,15,5,0,0,4,2,30,2,13,0,2,1,0,0,0,10,3,2,3,0,2,12,41,13,2,1,18,20,0,4,5,0,0,2,2,3,3,0,0,0,3,0,0,15,24,2,3,0,7,13,11,1,...,1,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,39,2,0,0,13,0,0,0,0,53,31,0,0,0,1,0,0,8,0,0,0,0,1,0,0,8,2,1,0,0,0,0,1,2,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,4,0,0,2,0,4,0,0,0,0,2,1,0,0,0,0,0,0,1,0,0,0,1,0,2,0,1
3,6,9,2,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,2,1,0,0,0,1,3,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,7,409,1,0,1,16,4,4,2,8,16,4,1,2,3,1,32,22,1,35,0,8,2,7,13,3,28,5,0,27,3,0,0,5,2,3,5,0,7,1,7,5,10,2,12,4,11,0,19,12,1,2,3,5,37,12,43,0,5,5,4,0,0,19,2,20,3,6,0,10,69,19,17,8,28,42,2,7,1,0,1,6,30,5,7,1,0,3,10,0,0,13,16,24,10,1,10,15,8,9,...,0,1,0,0,0,0,0,0,0,0,0,0,5,